Tool Used:
Google Colab + OpenAI Python SDK (Responses API)

Objective:
Build a 4-step prompt chain simulating a customer support workflow:

1. Issue classification

2. Missing information request

3. Resolution attempt

4. Escalation routing

Mode: Live (with API key) or Demo (without API key)

In [12]:
!pip -q install --upgrade openai

import os
from openai import OpenAI

MODEL = "gpt-4.1-mini"  # change if needed

LIVE_MODE = "OPENAI_API_KEY" in os.environ

if LIVE_MODE:
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    print("✅ LIVE MODE: Using OpenAI API")
else:
    client = None
    print("🟡 DEMO MODE: No OPENAI_API_KEY found, using sample outputs")

🟡 DEMO MODE: No OPENAI_API_KEY found, using sample outputs


In [13]:
import json

SYSTEM = """You are a customer support agent for an e-commerce company.

Tone: calm, friendly, concise.
Rules:
- Never promise refunds or specific delivery dates.
- Ask only for the minimum info needed.
- Follow the output format exactly as requested (JSON only when asked).
"""

CUSTOMER_MESSAGE = "My package says delivered but I never got it. I'm really frustrated."
CUSTOMER_FOLLOWUP_INFO = "Order #A12345. Delivery address is correct. No one in the household received it."

def ask(model: str, system: str, user: str) -> str:
    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
    )
    return resp.output_text

# ---------- Prompts (the “prompts you used” for your submission doc) ----------

STEP1_PROMPT = f"""
Task: Classify the customer's issue into one category:
["ORDER_STATUS","REFUND_RETURN","DAMAGED_ITEM","BILLING","ACCOUNT","TECHNICAL","OTHER"].

Also extract sentiment:
["angry","neutral","happy"].

Return JSON only in this schema:
{{
  "category": "...",
  "sentiment": "...",
  "key_facts": ["..."]
}}

Customer message:
{CUSTOMER_MESSAGE}
""".strip()

def step2_prompt(step1_json: str) -> str:
    return f"""
Given:
- Customer message: {CUSTOMER_MESSAGE}
- Step1 JSON: {step1_json}

Task:
1) Determine what critical info is missing to resolve the issue (max 2 items).
2) Write one short reply asking for that info.

Rules:
- Ask for order number if the category involves an order.
- Ask no more than 2 questions total.
- Be empathetic if sentiment is angry.

Return JSON only:
{{
  "missing_info": ["..."],
  "reply": "..."
}}
""".strip()

def step3_prompt(step1_json: str, step2_json: str) -> str:
    return f"""
Given:
- Customer message: {CUSTOMER_MESSAGE}
- Step1 JSON: {step1_json}
- Step2 JSON: {step2_json}

Assume the customer provided the missing info as:
{CUSTOMER_FOLLOWUP_INFO}

Task:
Provide a resolution attempt in 4-6 bullets.
Constraints:
- Do not mention internal policies.
- Do not promise a refund.
- If the issue requires human review, set needs_escalation true.

Return JSON only:
{{
  "resolution_bullets": ["..."],
  "needs_escalation": true/false,
  "escalation_reason": "... or null"
}}
""".strip()

def step4_prompt(step3_json: str) -> str:
    return f"""
Given Step3 JSON:
{step3_json}

Task:
If needs_escalation is true, produce:
- a routing target team: ["Logistics","Returns","Billing","Account","Fraud","Tech","Other"]
- a short internal note (2-4 bullets) summarizing the case clearly.

Return JSON only:
{{
  "route_to": "... or null",
  "internal_note": ["..."]
}}
If no escalation needed, set route_to to null and internal_note to [].
""".strip()

In [14]:
DEMO_STEP1 = json.dumps({
    "category": "ORDER_STATUS",
    "sentiment": "angry",
    "key_facts": [
        "Package marked delivered",
        "Customer did not receive the package"
    ]
}, indent=2)

DEMO_STEP2 = json.dumps({
    "missing_info": ["order_number", "delivery_address_confirmation"],
    "reply": "I’m sorry this happened — can you share your order number and confirm the delivery address so I can help right away?"
}, indent=2)

DEMO_STEP3 = json.dumps({
    "resolution_bullets": [
        "Confirm the delivery address and delivery date shown for Order #A12345.",
        "Check any safe drop-off locations around the address (front desk, mailroom, porch, side door).",
        "Ask household members/neighbors if they accepted it on your behalf.",
        "If it still can’t be located, I’ll open a delivery investigation with our shipping team.",
        "I’ll update you as soon as we get the carrier scan details or next steps."
    ],
    "needs_escalation": True,
    "escalation_reason": "Delivered scan reported but recipient did not receive; requires carrier investigation."
}, indent=2)

DEMO_STEP4 = json.dumps({
    "route_to": "Logistics",
    "internal_note": [
        "Order #A12345 shows delivered, customer reports not received.",
        "Customer confirmed address is correct; no household member received it.",
        "Request carrier scan details + GPS/photo proof if available; start investigation."
    ]
}, indent=2)

In [15]:
def run_chain():
    # Step 1
    if LIVE_MODE:
        step1 = ask(MODEL, SYSTEM, STEP1_PROMPT)
    else:
        step1 = DEMO_STEP1
    print("STEP 1 OUTPUT:\n", step1, "\n")

    # Step 2
    s2p = step2_prompt(step1)
    if LIVE_MODE:
        step2 = ask(MODEL, SYSTEM, s2p)
    else:
        step2 = DEMO_STEP2
    print("STEP 2 OUTPUT:\n", step2, "\n")

    # Step 3
    s3p = step3_prompt(step1, step2)
    if LIVE_MODE:
        step3 = ask(MODEL, SYSTEM, s3p)
    else:
        step3 = DEMO_STEP3
    print("STEP 3 OUTPUT:\n", step3, "\n")

    # Step 4
    s4p = step4_prompt(step3)
    if LIVE_MODE:
        step4 = ask(MODEL, SYSTEM, s4p)
    else:
        step4 = DEMO_STEP4
    print("STEP 4 OUTPUT:\n", step4, "\n")

run_chain()

STEP 1 OUTPUT:
 {
  "category": "ORDER_STATUS",
  "sentiment": "angry",
  "key_facts": [
    "Package marked delivered",
    "Customer did not receive the package"
  ]
} 

STEP 2 OUTPUT:
 {
  "missing_info": [
    "order_number",
    "delivery_address_confirmation"
  ],
  "reply": "I\u2019m sorry this happened \u2014 can you share your order number and confirm the delivery address so I can help right away?"
} 

STEP 3 OUTPUT:
 {
  "resolution_bullets": [
    "Confirm the delivery address and delivery date shown for Order #A12345.",
    "Check any safe drop-off locations around the address (front desk, mailroom, porch, side door).",
    "Ask household members/neighbors if they accepted it on your behalf.",
    "If it still can\u2019t be located, I\u2019ll open a delivery investigation with our shipping team.",
    "I\u2019ll update you as soon as we get the carrier scan details or next steps."
  ],
  "needs_escalation": true,
  "escalation_reason": "Delivered scan reported but recipient

How Prompt Chaining Works

Step 1 classifies the issue and extracts sentiment.

Step 2 uses Step 1’s JSON output to determine missing information.

Step 3 uses Step 1 and Step 2 outputs to generate a resolution attempt.

Step 4 evaluates Step 3 to determine whether escalation is required and routes the case.

Each step depends on the previous step’s structured output.